# comercio_ext_indices.tb_referencia_ncm_sh
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_SH_delta`
## 📌 Descrição do arquivo

Hierarquia **SH/HS** (SH2, SH4, SH6 e Seção romana) com descrições multilíngues.

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_SH6`|Código SH nível 6|Subposição internacional (6 dígitos)|
|`NO_SH6_POR`|Nome SH6 (PT)|Descrição SH6 em português|
|`NO_SH6_ESP`|Nome SH6 (ES)|Descrição SH6 em espanhol|
|`NO_SH6_ING`|Nome SH6 (EN)|Descrição SH6 em inglês|
|`CO_SH4`|Código SH nível 4|Posição (4 dígitos)|
|`NO_SH4_POR`|Nome SH4 (PT)|Descrição SH4 em português|
|`NO_SH4_ESP`|Nome SH4 (ES)|Descrição SH4 em espanhol|
|`NO_SH4_ING`|Nome SH4 (EN)|Descrição SH4 em inglês|
|`CO_SH2`|Código SH nível 2|Capítulo (2 dígitos)|
|`NO_SH2_POR`|Nome SH2 (PT)|Descrição do capítulo (PT)|
|`NO_SH2_ESP`|Nome SH2 (ES)|Descrição do capítulo (ES)|
|`NO_SH2_ING`|Nome SH2 (EN)|Descrição do capítulo (EN)|
|`CO_NCM_SECROM`|Código da seção (romano)|Macrogrupo do SH|
|`NO_SEC_POR`|Nome da seção (PT)|Nome macro da seção (PT)|
|`NO_SEC_ESP`|Nome da seção (ES)|Nome macro da seção (ES)|
|`NO_SEC_ING`|Nome da seção (EN)|Nome macro da seção (EN)|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta.tables import DeltaTable
jvm = spark._jvm
FileSystem = jvm.org.apache.hadoop.fs.FileSystem
Path = jvm.org.apache.hadoop.fs.Path
fs = FileSystem.get(spark._jsc.hadoopConfiguration())

In [0]:
bronzePath = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/ncm_sh/"
silverPath = "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_referencia_ncm_sh/"
silverTable = "silver_comercio_ext_indices.tb_referencia_ncm_sh"


In [0]:
silverSchema = T.StructType([
    T.StructField("CO_SH6",        T.StringType(),   False),
    T.StructField("NO_SH6_POR",    T.StringType(),   True ),
    T.StructField("NO_SH6_ESP",    T.StringType(),   True ),
    T.StructField("NO_SH6_ING",    T.StringType(),   True ),
    T.StructField("CO_SH4",        T.StringType(),   True ),
    T.StructField("NO_SH4_POR",    T.StringType(),   True ),
    T.StructField("NO_SH4_ESP",    T.StringType(),   True ),
    T.StructField("NO_SH4_ING",    T.StringType(),   True ),
    T.StructField("CO_SH2",        T.StringType(),   True ),
    T.StructField("NO_SH2_POR",    T.StringType(),   True ),
    T.StructField("NO_SH2_ESP",    T.StringType(),   True ),
    T.StructField("NO_SH2_ING",    T.StringType(),   True ),
    T.StructField("CO_NCM_SECROM", T.StringType(),   True ),
    T.StructField("NO_SEC_POR",    T.StringType(),   True ),
    T.StructField("NO_SEC_ESP",    T.StringType(),   True ),
    T.StructField("NO_SEC_ING",    T.StringType(),   True ),
    T.StructField("TS_REF",        T.TimestampType(),False),
    T.StructField("NM_ORIGEM",     T.StringType(),   False),
])

## Extração
> #### **saprk.read**

In [0]:
df_bronze = spark.read.format("delta").load(bronzePath)

##Normalização
> #### **datatype**

In [0]:
def up(s): return F.upper(F.trim(F.col(s).cast(T.StringType())))

df_norm = (
    df_bronze
    .withColumn("CO_SH6", up("CO_SH6"))
    .withColumn("NO_SH6_POR", F.col("NO_SH6_POR").cast(T.StringType()))
    .withColumn("NO_SH6_ESP", F.col("NO_SH6_ESP").cast(T.StringType()))
    .withColumn("NO_SH6_ING", F.col("NO_SH6_ING").cast(T.StringType()))
    .withColumn("CO_SH4", up("CO_SH4"))
    .withColumn("NO_SH4_POR", F.col("NO_SH4_POR").cast(T.StringType()))
    .withColumn("NO_SH4_ESP", F.col("NO_SH4_ESP").cast(T.StringType()))
    .withColumn("NO_SH4_ING", F.col("NO_SH4_ING").cast(T.StringType()))
    .withColumn("CO_SH2", up("CO_SH2"))
    .withColumn("NO_SH2_POR", F.col("NO_SH2_POR").cast(T.StringType()))
    .withColumn("NO_SH2_ESP", F.col("NO_SH2_ESP").cast(T.StringType()))
    .withColumn("NO_SH2_ING", F.col("NO_SH2_ING").cast(T.StringType()))
    .withColumn("CO_NCM_SECROM", up("CO_NCM_SECROM"))
    .withColumn("NO_SEC_POR", F.col("NO_SEC_POR").cast(T.StringType()))
    .withColumn("NO_SEC_ESP", F.col("NO_SEC_ESP").cast(T.StringType()))
    .withColumn("NO_SEC_ING", F.col("NO_SEC_ING").cast(T.StringType()))
    .withColumn("TS_REF", F.current_timestamp())
    .withColumn("NM_ORIGEM", F.lit("/landingbeca2026jan/balancacomercial/ncm_sh"))
)

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = df_norm.filter(F.col("CO_SH6").isNotNull())



In [0]:
df_dedup = df_valid.dropDuplicates(["CO_SH6"])

In [0]:
df_silver = df_dedup.select(
    "CO_SH6","NO_SH6_POR","NO_SH6_ESP","NO_SH6_ING",
    "CO_SH4","NO_SH4_POR","NO_SH4_ESP","NO_SH4_ING",
    "CO_SH2","NO_SH2_POR","NO_SH2_ESP","NO_SH2_ING",
    "CO_NCM_SECROM","NO_SEC_POR","NO_SEC_ESP","NO_SEC_ING",
    "TS_REF","NM_ORIGEM"
)

##Carga
> #### **merge**

In [0]:
delta_target = DeltaTable.forName(spark, "silver_comercio_ext_estatisticas.tb_referencia_ncm_sh")

merge_condition = """
  t.CO_SH6 = s.CO_SH6
"""

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "NO_SH6_POR": "s.NO_SH6_POR",
        "NO_SH6_ESP": "s.NO_SH6_ESP",
        "NO_SH6_ING": "s.NO_SH6_ING",
        "CO_SH4": "s.CO_SH4",
        "NO_SH4_POR": "s.NO_SH4_POR",
        "NO_SH4_ESP": "s.NO_SH4_ESP",
        "NO_SH4_ING": "s.NO_SH4_ING",
        "CO_SH2": "s.CO_SH2",
        "NO_SH2_POR": "s.NO_SH2_POR",
        "NO_SH2_ESP": "s.NO_SH2_ESP",
        "NO_SH2_ING": "s.NO_SH2_ING",
        "CO_NCM_SECROM": "s.CO_NCM_SECROM",
        "NO_SEC_POR": "s.NO_SEC_POR",
        "NO_SEC_ESP": "s.NO_SEC_ESP",
        "NO_SEC_ING": "s.NO_SEC_ING",
        "TS_REF": "s.TS_REF",
        "NM_ORIGEM": "s.NM_ORIGEM",
    })
    .whenNotMatchedInsertAll()
    .execute()
)